In [1]:
from urllib.request import urlopen
import timm
import torch
import zipfile,os
from PIL import Image
from pathlib import Path
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm  # Import tqdm for progress bar
import os

c:\ProgramData\anaconda3\envs\ultralytics_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

In [3]:
os.path.join(os.getcwd(), 'GCD.zip')

'c:\\In AI We Trust\\GCD.zip'

In [4]:
local_zip = os.path.join(os.getcwd(), 'GCD.zip')
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(os.getcwd())
zip_ref.close()

In [5]:
#train_dir = os.path.join(os.getcwd(), 'GCD', 'train')
test_dir = os.path.join(os.getcwd(), 'GCD', 'test')

print(test_dir)

c:\In AI We Trust\GCD\test


In [6]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.version.cuda)         # Should match the installed CUDA version

True
11.8


In [ ]:
model = timm.create_model(
    'maxvit_small_tf_224.in1k',
    pretrained=True,
    num_classes=7,
)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

In [8]:
'''
train_dir = Path(train_dir)

for img_path in train_dir.rglob("*.*"):  # Recursively find all files
    if img_path.suffix.lower() in [".jpg", ".jpeg", ".png"]:  # Check for valid image extensions
        img = Image.open(img_path).convert("RGB")  # Open the image and convert to RGB
        img_tensor = transforms(img).unsqueeze(0)  # Preprocess and add batch dimension

        # Pass the image through the model
        output = model.forward_features(transforms(img_tensor))

        print(f"Processed {img_path}: Output Shape - {output[-1].shape if isinstance(output, list) else output.shape}")
'''

'\ntrain_dir = Path(train_dir)\n\nfor img_path in train_dir.rglob("*.*"):  # Recursively find all files\n    if img_path.suffix.lower() in [".jpg", ".jpeg", ".png"]:  # Check for valid image extensions\n        img = Image.open(img_path).convert("RGB")  # Open the image and convert to RGB\n        img_tensor = transforms(img).unsqueeze(0)  # Preprocess and add batch dimension\n\n        # Pass the image through the model\n        output = model.forward_features(transforms(img_tensor))\n\n        print(f"Processed {img_path}: Output Shape - {output[-1].shape if isinstance(output, list) else output.shape}")\n'

In [9]:

# for data in train_loader:
#     print(data)  # This will show you the structure of the data being returned
#     inputs, targets = data  # Unpack only if it has the correct structure

In [11]:
from torchvision import transforms


class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_names = os.listdir(data_dir)

        for label, class_name in enumerate(self.class_names):
            class_dir = os.path.join(data_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.images.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label  # Ensure this returns a tuple of (image, label)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Create an instance of the CustomDataset
dataset = CustomDataset(data_dir=test_dir, transform=transform)

train_loader = DataLoader(dataset, batch_size=8, shuffle=True)
for data in train_loader:
    print(data[0].shape)  # This will show you the structure of the data being returned
    inputs, targets = data  # Unpack only if it has the correct structure
    break

torch.Size([8, 3, 224, 224])


In [12]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda")
model.to(device)
num_epochs = 2

# Training loop
model.train()  # Set the model to training mode
for epoch in range(num_epochs):
    epoch_loss = 0  # To accumulate loss for the epoch
    progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")  # Progress bar
    
    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)  # Move to device

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        # Update progress bar description with the current loss
        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Avg Loss: {avg_loss:.4f}")

Epoch [1/2]: 100%|██████████| 1125/1125 [33:16<00:00,  1.77s/it, loss=1.03] 


Epoch [1/2], Avg Loss: 1.0363


Epoch [2/2]: 100%|██████████| 1125/1125 [33:29<00:00,  1.79s/it, loss=0.259]

Epoch [2/2], Avg Loss: 0.7303


In [13]:
test_dataset = ImageFolder(root=test_dir, transform=transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 4. Define testing loop
correct_predictions = 0
total_samples = 0
all_targets = []
all_preds = []

'''
with torch.no_grad():  # Disable gradient computation for inference
    for inputs, targets in test_loader:
        # Perform forward pass
        outputs = model(inputs)

        # Get predicted class indices
        _, preds = torch.max(outputs, dim=1)

        # Compare predictions with ground truth
        correct_predictions += (preds == targets).sum().item()
        total_samples += targets.size(0)

        # Store predictions and labels for metrics
        all_targets.extend(targets.tolist())
        all_preds.extend(preds.tolist())
'''
# Perform forward pass
outputs = model(inputs)

# Get predicted class indices
_, preds = torch.max(outputs, dim=1)

# Compare predictions with ground truth
correct_predictions += (preds == targets).sum().item()
total_samples += targets.size(0)

# Store predictions and labels for metrics
all_targets.extend(targets.tolist())
all_preds.extend(preds.tolist())

accuracy = accuracy_score(all_targets, all_preds)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 100.00%


Pipeline

In [14]:
modelName = ['maxvit_small_tf_224.in1k', 'maxvit_rmlp_base_rw_224.sw_in12k_ft_in1k', 'maxvit_large_tf_224.in1k']

for x in modelName:
    # choose model
    model = timm.create_model(
        x,
        pretrained=True,
        num_classes=7,
    )
    model = model.eval()

    # get model specific transforms (normalization, resize)
    data_config = timm.data.resolve_model_data_config(model)
    transforms = timm.data.create_transform(**data_config, is_training=False)



    from torchvision import transforms

    class CustomDataset(Dataset):
        def __init__(self, data_dir, transform=None):
            self.data_dir = data_dir
            self.transform = transform
            self.images = []
            self.labels = []
            self.class_names = os.listdir(data_dir)

            for label, class_name in enumerate(self.class_names):
                class_dir = os.path.join(data_dir, class_name)
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    self.images.append(img_path)
                    self.labels.append(label)

        def __len__(self):
            return len(self.images)

        def __getitem__(self, idx):
            img_path = self.images[idx]
            image = Image.open(img_path).convert('RGB')
            label = self.labels[idx]

            if self.transform:
                image = self.transform(image)

            return image, label  # Ensure this returns a tuple of (image, label)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to 224x224
        transforms.ToTensor(),           # Convert images to PyTorch tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
    ])

    # Create an instance of the CustomDataset
    dataset = CustomDataset(data_dir=test_dir, transform=transform)

    train_loader = DataLoader(dataset, batch_size=8, shuffle=True)
    for data in train_loader:
        print(data[0].shape)  # This will show you the structure of the data being returned
        inputs, targets = data  # Unpack only if it has the correct structure
        break



    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = torch.device("cuda")
    model.to(device)
    num_epochs = 2

    # Training loop
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        epoch_loss = 0  # To accumulate loss for the epoch
        progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")  # Progress bar
        
        for inputs, targets in progress_bar:
            inputs, targets = inputs.to(device), targets.to(device)  # Move to device

            optimizer.zero_grad()  # Zero the gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, targets)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            # Update progress bar description with the current loss
            epoch_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Avg Loss: {avg_loss:.4f}")



    test_dataset = ImageFolder(root=test_dir, transform=transforms)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # 4. Define testing loop
    correct_predictions = 0
    total_samples = 0
    all_targets = []
    all_preds = []

    '''
    with torch.no_grad():  # Disable gradient computation for inference
        for inputs, targets in test_loader:
            # Perform forward pass
            outputs = model(inputs)

            # Get predicted class indices
            _, preds = torch.max(outputs, dim=1)

            # Compare predictions with ground truth
            correct_predictions += (preds == targets).sum().item()
            total_samples += targets.size(0)

            # Store predictions and labels for metrics
            all_targets.extend(targets.tolist())
            all_preds.extend(preds.tolist())
    '''
    # Perform forward pass
    outputs = model(inputs)

    # Get predicted class indices
    _, preds = torch.max(outputs, dim=1)

    # Compare predictions with ground truth
    correct_predictions += (preds == targets).sum().item()
    total_samples += targets.size(0)

    # Store predictions and labels for metrics
    all_targets.extend(targets.tolist())
    all_preds.extend(preds.tolist())

    # Print accuracy
    accuracy = accuracy_score(all_targets, all_preds)
    print(x)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    import os
    with open("accuracy.txt", "a") as file:
        file.write("accuracy score of ", x, " : ")
        file.write(accuracy, "\n")

torch.Size([8, 3, 224, 224])


Epoch [1/2]: 100%|██████████| 1125/1125 [1:30:39<00:00,  4.83s/it, loss=0.677]


Epoch [1/2], Avg Loss: 1.0315


Epoch [2/2]: 100%|██████████| 1125/1125 [1:31:33<00:00,  4.88s/it, loss=0.702]


Epoch [2/2], Avg Loss: 0.7358
maxvit_small_tf_224.in1k
Accuracy: 87.50%


TypeError: TextIOWrapper.write() takes exactly one argument (3 given)